<a href="https://colab.research.google.com/github/nickxr/CudaLabs/blob/main/Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Thu Sep  9 18:19:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
torch.cuda.is_available()
# Output would be True if Pytorch is using GPU otherwise it would be False.

True

Полностью удалим все предыдущие версии CUDA. Нам нужно обновить облачный экземпляр CUDA.

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Установим CUDA версии 9 

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

Проверим, какая версия CUDA у нас установлена

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


# New Section

Установим расширение CUDA для jupyter

In [4]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-t6ndrh0e
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-t6ndrh0e
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4305 sha256=abe6db6e9da7926f87b212b578657f7b5fc2881128456a8409b820bb7f909c8a
  Stored in directory: /tmp/pip-ephem-wheel-cache-ecpwf28c/wheels/c5/2b/c0/87008e795a14bbcdfc7c846a00d06981916331eb980b6c8bdf
Successfully built NVCCPlugin


Загрузим расширение

In [5]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


Напишем Hello World, чтобы проверить, что плагин работает

In [ ]:
%%cu
#include <iostream>
    int
    main()
{
    std::cout << "Hello World\n";
    return 0;
}

Hello World



Задание 7. Вычислить функцию синус икс делить на икс.

In [47]:
%%cu
#include <stdio.h>
#include <iostream>
#include <cuda_runtime.h>
#include <chrono>
#define N 256

__global__ void myKernel(double *x,double *res)
{
  int index = blockIdx.x * blockDim.x + threadIdx.x;
  res[index] = sin(x[index]) / x[index];
}

int main(int argc, char** argv) {

double x[N],res[N];

double *dev_x,*dev_res;

cudaMalloc((void**)&dev_x, N * sizeof(int));
cudaMalloc((void**)&dev_res, N * sizeof(int));

auto t1 = std::chrono::high_resolution_clock::now();

for (int i = 0; i < N; ++i) {
  res[i] = sin(i) / i;
}
auto t2 = std::chrono::high_resolution_clock::now();

auto duration = std::chrono::duration_cast<std::chrono::microseconds>( t2 - t1 ).count();

std::cout <<"Time CPU: "<< duration<<"ms\n";

cudaMemcpy(dev_x, &x, N * sizeof(int), cudaMemcpyHostToDevice);
cudaMemcpy(dev_res, &res, N * sizeof(int), cudaMemcpyHostToDevice);

t1 = std::chrono::high_resolution_clock::now();

myKernel<<<1,N>>>(dev_x, dev_res);
cudaDeviceSynchronize();

t2 = std::chrono::high_resolution_clock::now();

duration = std::chrono::duration_cast<std::chrono::microseconds>( t2 - t1 ).count();

std::cout <<"Time GPU: "<< duration<<"ms\n";

std::cout <<"Results\n";

for (int i = 0; i < N; ++i) {
  res[i] = sin(i) / i;
  printf("For index %d value =  %f\n", i, res[i]);
}

cudaFree(dev_x);
cudaFree(dev_res);
return 0;
}

Time CPU: 22ms
Time GPU: 8ms
Results
For index 0 value =  -nan
For index 1 value =  0.841471
For index 2 value =  0.454649
For index 3 value =  0.047040
For index 4 value =  -0.189201
For index 5 value =  -0.191785
For index 6 value =  -0.046569
For index 7 value =  0.093855
For index 8 value =  0.123670
For index 9 value =  0.045791
For index 10 value =  -0.054402
For index 11 value =  -0.090908
For index 12 value =  -0.044714
For index 13 value =  0.032321
For index 14 value =  0.070758
For index 15 value =  0.043353
For index 16 value =  -0.017994
For index 17 value =  -0.056553
For index 18 value =  -0.041722
For index 19 value =  0.007888
For index 20 value =  0.045647
For index 21 value =  0.039841
For index 22 value =  -0.000402
For index 23 value =  -0.036792
For index 24 value =  -0.037732
For index 25 value =  -0.005294
For index 26 value =  0.029329
For index 27 value =  0.035421
For index 28 value =  0.009675
For index 29 value =  -0.022884
For index 30 value =  -0.032934
F